In [22]:
pip install prince

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
random.seed(9)

In [2]:
train = pd.read_csv('../input/machinehack-ml-merchandise-popularity-prediction/MPP_Dataset/Train.csv')
test  = pd.read_csv('../input/machinehack-ml-merchandise-popularity-prediction/MPP_Dataset/Test.csv')

In [3]:
train.drop_duplicates(inplace=True);

In [4]:
train['Merged'] = train[train.columns[0:11]].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
test['Merged'] = test[test.columns[0:11]].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

In [5]:
test = test.merge(train,left_on='Merged',right_on='Merged',how='left',suffixes=('', '_y'))

In [6]:
# A unique pattern was observed in Train set,if Store_Ratio is zero then its populartiy is zero. 
#So adding this to test set to improve the overall score of the model
test.loc[(test.Store_Ratio == 0 ),'popularity']= 0

In [7]:
test[test.Store_Ratio == 0 ]

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,...,Category_1_y,Store_Score_y,Category_2_y,Store_Presence_y,Score_1_y,Score_2_y,Score_3_y,Score_4_y,time_y,popularity
358,0.0,0.044200,0,-16.383,1,0.948000,0.00181,0.2350,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2873,0.0,0.009830,10,-14.866,0,0.648000,0.90600,0.1080,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5982,0.0,0.080600,11,-18.483,1,0.477000,0.97000,0.9720,0.0,0.0,...,11.0,-18.483,1.0,0.477,0.970,0.9720,0.0,0.0,169902.0,0.0
5984,0.0,0.000081,2,-27.064,1,0.000171,0.97700,0.4360,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
6306,0.0,0.018100,4,-30.676,0,0.996000,0.84200,0.0795,0.0,0.0,...,4.0,-30.676,0.0,0.996,0.842,0.0795,0.0,0.0,129250.0,0.0
7565,0.0,0.642000,2,-16.014,1,0.120000,1.00000,0.5240,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7761,0.0,0.114000,0,-20.603,1,0.976000,0.71300,0.0725,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
8615,0.0,0.009830,10,-14.866,0,0.648000,0.90600,0.1080,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
8829,0.0,0.000000,0,-43.184,0,0.000000,0.00000,0.0000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
11152,0.0,0.003560,1,-27.949,1,0.993000,0.96300,0.1040,0.0,0.0,...,1.0,-27.949,1.0,0.993,0.963,0.1040,0.0,0.0,101625.0,0.0


In [8]:
#Remove unwanted columns from the mergered dataset 

In [9]:
test = test[['Store_Ratio', 'Basket_Ratio', 'Category_1', 'Store_Score',
       'Category_2', 'Store_Presence', 'Score_1', 'Score_2', 'Score_3',
       'Score_4', 'time','popularity']]
test_predeicted = test.pop('popularity');

In [10]:
len(test)

12140

In [11]:
train.pop('Merged');

In [12]:
train.head()

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time,popularity
0,0.407,0.00380,2,-35.865,1,0.9920,0.944,0.0988,0.1100,113.911,189125,4
1,0.234,0.10500,0,-19.884,1,0.9440,0.900,0.1290,0.0382,76.332,186513,4
2,0.668,0.72600,9,-7.512,1,0.0387,0.000,0.1340,0.4530,124.075,172143,4
3,0.184,0.00561,4,-34.357,1,0.8490,0.931,0.1110,0.0641,79.037,184000,5
4,0.231,0.13100,6,-22.842,1,0.9370,0.000,0.1090,0.0677,109.560,186507,3


In [13]:
train['popularity'].value_counts()

4    12850
3     1784
5      376
1      261
0       14
Name: popularity, dtype: int64

In [14]:
train.head()

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time,popularity
0,0.407,0.00380,2,-35.865,1,0.9920,0.944,0.0988,0.1100,113.911,189125,4
1,0.234,0.10500,0,-19.884,1,0.9440,0.900,0.1290,0.0382,76.332,186513,4
2,0.668,0.72600,9,-7.512,1,0.0387,0.000,0.1340,0.4530,124.075,172143,4
3,0.184,0.00561,4,-34.357,1,0.8490,0.931,0.1110,0.0641,79.037,184000,5
4,0.231,0.13100,6,-22.842,1,0.9370,0.000,0.1090,0.0677,109.560,186507,3


In [15]:
train.describe()

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time,popularity
count,15285.000000,15285.000000,15285.000000,15285.000000,15285.000000,15285.000000,15285.000000,15285.000000,15285.000000,15285.000000,1.528500e+04,15285.000000
mean,0.548290,0.498163,5.137782,-11.741641,0.646189,0.463499,0.303094,0.165802,0.430720,115.912332,2.222829e+05,3.852993
std,0.201223,0.297053,3.542558,7.989707,0.478167,0.376327,0.405763,0.136753,0.273821,31.186151,1.011309e+05,0.536710
min,0.000000,0.000216,0.000000,-47.576000,0.000000,0.000000,0.000000,0.011900,0.000000,0.000000,3.184400e+04,0.000000
25%,0.419000,0.232000,2.000000,-15.338000,0.000000,0.079200,0.000000,0.094600,0.193000,91.990000,1.726670e+05,4.000000
50%,0.577000,0.532000,5.000000,-8.977000,1.000000,0.404000,0.001490,0.113000,0.405000,114.991000,2.079000e+05,4.000000
75%,0.701000,0.749000,8.000000,-5.893000,1.000000,0.871000,0.833000,0.180000,0.653000,135.027000,2.512680e+05,4.000000
max,0.998000,1.000000,11.000000,-0.079000,1.000000,0.996000,1.000000,0.991000,0.999000,219.701000,3.908467e+06,5.000000


In [16]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train['Score_4'] = scaler.fit_transform(train[['Score_4']])
test['Score_4'] = scaler.transform(test[['Score_4']])

scaler = MinMaxScaler()
train['Store_Score'] = scaler.fit_transform(train[['Store_Score']])
test['Store_Score'] = scaler.transform(test[['Store_Score']])

train['time'] = scaler.fit_transform(train[['time']])
test['time'] =  scaler.transform(test[['time']])

In [17]:
test.isnull().sum()

Store_Ratio       0
Basket_Ratio      0
Category_1        0
Store_Score       0
Category_2        0
Store_Presence    0
Score_1           0
Score_2           0
Score_3           0
Score_4           0
time              0
dtype: int64

In [18]:
#Featuer Engineering but it didnt work for overall result 

In [19]:
#import prince
#pca = prince.PCA(n_components=2)
#train[['PCA1','PCA2']] = pca.fit_transform(train[['Score_1','Score_2','Score_3','Score_4']])
#test[['PCA1','PCA2']]  = pca.transform(test[['Score_1','Score_2','Score_3','Score_4']])

In [20]:
#ordered_labels = []
#ordered_labels  = train.groupby(['Category_1'])['popularity'].mean()
#train['avg_rat'] = train['Category_1'].map(ordered_labels)
#test['avg_rat'] = test['Category_1'].map(ordered_labels)

#train['Cat1&2'] = train['Category_1'].astype('str') + train['Category_2'].astype('str')
#test['Cat1&2'] = test['Category_1'].astype('str') + test['Category_2'].astype('str')

#ordered_labels = []
#ordered_labels  = train.groupby(['Cat1&2'])['popularity'].mean()
#train['Cat1&2'] = train['Cat1&2'].map(ordered_labels)
#test['Cat1&2'] = test['Cat1&2'].map(ordered_labels)

In [21]:
from sklearn.model_selection import cross_val_score

def cross_valid(model,X,Y,cv=10):
    results = cross_val_score(model, X, Y,scoring="neg_log_loss",cv=cv,n_jobs=-1)
    return (results)

In [22]:
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier,VotingClassifier
from xgboost import XGBClassifier,XGBRFClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from imblearn.ensemble import BalancedBaggingClassifier,BalancedRandomForestClassifier,EasyEnsembleClassifier

In [23]:
Y = train.pop('popularity')
X = train

In [24]:
X.columns

Index(['Store_Ratio', 'Basket_Ratio', 'Category_1', 'Store_Score',
       'Category_2', 'Store_Presence', 'Score_1', 'Score_2', 'Score_3',
       'Score_4', 'time'],
      dtype='object')

In [25]:
estimators = [('LR',LogisticRegression()),('SVC',ExtraTreesClassifier(random_state=9)),
              ('CB',CatBoostClassifier(loss_function='MultiClass',random_state=9)),
              ('GR',GradientBoostingClassifier(random_state=9))]

Reg_Models = [LogisticRegression(),#KNeighborsClassifier(),
              OneVsRestClassifier(SVC(probability=True)),
              GradientBoostingClassifier(random_state=9),
              #DecisionTreeClassifier(),ExtraTreeClassifier(),
              #RandomForestClassifier(),
              ExtraTreesClassifier(random_state=9),
              XGBClassifier(objective='multi:softmax',random_state=9),
              CatBoostClassifier(loss_function='MultiClass',random_state=9),
              LGBMClassifier(objective = 'multiclass',random_state=9),
              #StackingClassifier(estimators=estimators)
              VotingClassifier(estimators=estimators,n_jobs=-1,voting='soft')
              #BalancedBaggingClassifier(),BalancedRandomForestClassifier(),EasyEnsembleClassifier()
             ]


for i in Reg_Models:
  model = i
  error = cross_valid(model,X,Y,cv=10)

  print(str(model).split("(")[0], error)
  print(error.mean())

LogisticRegression [-0.47515758 -0.47982603 -0.47007765 -0.47824807 -0.47340766 -0.48544801
 -0.48538004 -0.47419751 -0.46923674 -0.47111609]
-0.4762095365713167


/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


OneVsRestClassifier [-0.55418817 -0.55775346 -0.55916172 -0.55137994 -0.55228328 -0.54444308
 -0.55025466 -0.56262239 -0.55008447 -0.55138082]
-0.5533551983757588


/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GradientBoostingClassifier [-0.4697925  -0.49193782 -0.46698124 -0.47102498 -0.46598901 -0.47426642
 -0.47351789 -0.47364215 -0.47393355 -0.47797029]
-0.4739055843467586
ExtraTreesClassifier [-0.69456232 -0.65635581 -0.69547485 -0.56702852 -0.6815804  -0.63679733
 -0.65921456 -0.69206647 -0.60509395 -0.66933434]
-0.6557508543328694
XGBClassifier [-0.51068379 -0.49318824 -0.51254962 -0.48708858 -0.49748502 -0.48952637
 -0.5109934  -0.49019449 -0.47550804 -0.50457287]
-0.49717904332263546


/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


<catboost.core.CatBoostClassifier object at 0x7f89ec50b550> [-0.49022276 -0.47268652 -0.47934949 -0.4710375  -0.48041396 -0.47424203
 -0.48914499 -0.47738987 -0.46192446 -0.48362984]
-0.4780041412943839
LGBMClassifier [-2.33585917 -1.09273477 -0.75615867 -0.61591618 -0.64755525 -0.48792327
 -0.65759322 -0.57537462 -0.45714611 -0.72175558]
-0.8348016855318521
VotingClassifier [-0.46368051 -0.45773083 -0.45840141 -0.45697528 -0.45815817 -0.46176551
 -0.47080564 -0.46432101 -0.44929069 -0.46407297]
-0.46052020121171006


In [ ]:
#Hyperparamter Tunning didnt work

In [ ]:
#Featuer selection didnt work for individual models

In [26]:
from sklearn.model_selection import cross_validate

estimators = [('LR',LogisticRegression()),('SVC',ExtraTreesClassifier(random_state=9)),
              ('CB',CatBoostClassifier(loss_function='MultiClass',random_state=9)),
              ('GR',GradientBoostingClassifier(random_state=9))]

clf=VotingClassifier(estimators=estimators,n_jobs=-1,voting='soft')

output = cross_validate(clf,X,Y,n_jobs=-1, cv=10, scoring = 'neg_log_loss', return_estimator =True,return_train_score=True)

In [27]:
output

{'fit_time': array([91.06550241, 91.68117714, 91.45545578, 91.64695477, 91.48189712,
        91.51293993, 89.98979163, 89.37923717, 45.30061531, 44.32419634]),
 'score_time': array([0.14627504, 0.15946031, 0.14550066, 0.15733075, 0.13683128,
        0.20922112, 0.17455029, 0.37324858, 0.11384559, 0.09506536]),
 'estimator': (VotingClassifier(estimators=[('LR', LogisticRegression()),
                               ('SVC', ExtraTreesClassifier(random_state=9)),
                               ('CB',
                                <catboost.core.CatBoostClassifier object at 0x7f89ec477890>),
                               ('GR',
                                GradientBoostingClassifier(random_state=9))],
                   n_jobs=-1, voting='soft'),
  VotingClassifier(estimators=[('LR', LogisticRegression()),
                               ('SVC', ExtraTreesClassifier(random_state=9)),
                               ('CB',
                                <catboost.core.CatBoostClassifier

In [28]:
test.head()

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time
0,0.164,0.994,1,0.502305,0,0.12400,0.99100,0.7890,0.00001,0.580798,0.052776
1,0.586,0.636,4,0.839337,1,0.00208,0.32200,0.1070,0.35500,0.620557,0.057211
2,0.457,0.743,0,0.841843,1,0.40400,0.00931,0.2010,0.68800,0.705058,0.047419
3,0.423,0.748,5,0.794661,1,0.03000,0.22100,0.1690,0.42800,0.427750,0.075673
4,0.802,0.756,2,0.774470,0,0.08430,0.00765,0.0521,0.96300,0.599519,0.066153


In [29]:
final= []
sub_file = []
for idx,estimator in enumerate(output['estimator']):
  print(idx)
  print(estimator)
  score = estimator.predict_proba(test)
  sub_file.append(score) 

0
VotingClassifier(estimators=[('LR', LogisticRegression()),
                             ('SVC', ExtraTreesClassifier(random_state=9)),
                             ('CB',
                              <catboost.core.CatBoostClassifier object at 0x7f89ec477890>),
                             ('GR',
                              GradientBoostingClassifier(random_state=9))],
                 n_jobs=-1, voting='soft')
1
VotingClassifier(estimators=[('LR', LogisticRegression()),
                             ('SVC', ExtraTreesClassifier(random_state=9)),
                             ('CB',
                              <catboost.core.CatBoostClassifier object at 0x7f89ec07d890>),
                             ('GR',
                              GradientBoostingClassifier(random_state=9))],
                 n_jobs=-1, voting='soft')
2
VotingClassifier(estimators=[('LR', LogisticRegression()),
                             ('SVC', ExtraTreesClassifier(random_state=9)),
                       

In [30]:
#taking mean of all 5 models and creating dataframe
final = np.mean(sub_file,0)
final_df = pd.DataFrame(final)

In [31]:
final_df.head()

,0,1,2,3,4
0,0.001555,0.028927,0.549099,0.368929,0.051489
1,0.000040,0.004808,0.033800,0.953407,0.007946
2,0.000044,0.005942,0.065808,0.914261,0.013945
3,0.000070,0.006093,0.044532,0.936576,0.012730
4,0.000007,0.000810,0.008301,0.988476,0.002405


In [32]:
#Moving the predection vlaue of record's duplicate in Test from Train set
final_df['pred'] = test_predeicted

In [33]:
final_df.head()

,0,1,2,3,4,pred
0,0.001555,0.028927,0.549099,0.368929,0.051489,3.0
1,0.000040,0.004808,0.033800,0.953407,0.007946,NaN
2,0.000044,0.005942,0.065808,0.914261,0.013945,NaN
3,0.000070,0.006093,0.044532,0.936576,0.012730,NaN
4,0.000007,0.000810,0.008301,0.988476,0.002405,4.0


In [34]:
#Simple logic to make the probablity 100% for record which are duplicate from Train for test

In [35]:
final_df.loc[(final_df.pred == 0 ),0]= 1
final_df.loc[(final_df.pred == 0 ),1]= 0
final_df.loc[(final_df.pred == 0 ),2]= 0
final_df.loc[(final_df.pred == 0 ),3]= 0
final_df.loc[(final_df.pred == 0 ),4]= 0

final_df.loc[(final_df.pred == 1 ),0]= 0
final_df.loc[(final_df.pred == 1 ),1]= 1
final_df.loc[(final_df.pred == 1 ),2]= 0
final_df.loc[(final_df.pred == 1 ),3]= 0
final_df.loc[(final_df.pred == 1 ),4]= 0


final_df.loc[(final_df.pred == 3 ),0]= 0
final_df.loc[(final_df.pred == 3 ),1]= 0
final_df.loc[(final_df.pred == 3 ),2]= 1
final_df.loc[(final_df.pred == 3 ),3]= 0
final_df.loc[(final_df.pred == 3 ),4]= 0

final_df.loc[(final_df.pred == 4 ),0]= 0
final_df.loc[(final_df.pred == 4 ),1]= 0
final_df.loc[(final_df.pred == 4 ),2]= 0
final_df.loc[(final_df.pred == 4 ),3]= 1
final_df.loc[(final_df.pred == 4 ),4]= 0

final_df.loc[(final_df.pred == 5 ),0]= 0
final_df.loc[(final_df.pred == 5 ),1]= 0
final_df.loc[(final_df.pred == 5 ),2]= 0
final_df.loc[(final_df.pred == 5 ),3]= 0
final_df.loc[(final_df.pred == 5 ),4]= 1


In [36]:
#Drop unwanted colu,mns from final submission
final_df.pop('pred');

In [37]:
final_df.head()

,0,1,2,3,4
0,0.000000,0.000000,1.000000,0.000000,0.000000
1,0.000040,0.004808,0.033800,0.953407,0.007946
2,0.000044,0.005942,0.065808,0.914261,0.013945
3,0.000070,0.006093,0.044532,0.936576,0.012730
4,0.000000,0.000000,0.000000,1.000000,0.000000


In [38]:
final_df.to_csv('Voting_Reg_last.csv',index=False)